In [1]:
import numpy as np

from proj1_helpers import load_csv_data, create_csv_submission
from feature_selection import select_best_degrees, build_poly_by_feature
from feature_selection import compute_log, compute_theta, compute_physics
from correction_rate import cross_validation, print_score
from evaluation import predict_regression_labels

%load_ext autoreload
%autoreload 2

In [3]:
############################
## Training
############################
print('training started')

dat_dir = '../data/'
# load the training set
print('loading the training dataset...')
y_train_pre, tx_train, ids_train = load_csv_data(dat_dir + "train.csv", sub_sample=False)
print('data loaded...')

training started
loading the training dataset...
data loaded...


In [4]:
# construct the featues using log()
index_log = [0,1,2,4,5,6,7,9,10,12,16,21,23,24,25,26,27,28,29]
tx_log,mean_log,std_log = compute_log(tx_train, index_log)

# construct the featues using cosine()
index_theta = [14,15,17,18,20]
tx_theta,mean_theta,std_theta = compute_theta(tx_train,index_theta)

# construct the featues with physics meanings: 
# index_physics_A (mass) * index_physics_B / index_physics_C
index_physics_A = [0]
index_physics_B = [10,13,13,9]
index_physics_C = [9,16,21,10]
tx_physics, mean_physics, std_physics = compute_physics(tx_train,
                                                        index_physics_A,
                                                        index_physics_B,
                                                        index_physics_C)

# combine all the selected features for training set
train_new = np.c_[tx_log, tx_theta, tx_physics]

In [5]:
best_degrees = np.load('best_degrees.npy')
# reconstruct all the features at their own best degrees
train_best_degrees = build_poly_by_feature(train_new, best_degrees)

In [8]:
y_train=y_train_pre.reshape(y_train_pre.shape[0],1)
# innitialize hyper parameters
k_fold = 15
method = 'RR'
X = train_best_degrees
y = np.copy(y_train)

initial_w_pre = np.zeros((np.size(X,1)))
initial_w=initial_w_pre.reshape(initial_w_pre.shape[0],1)

## Least squares

In [36]:
# Least squares
scores, weights,loss_avg , score_avg = cross_validation(y, X_sigmoid, 'LS', {},
                                                        threshold = 0,
                                                         k=k_fold, seed=0)
print_score(loss_avg, score_avg, scores, method='Least squares')

                       Least squares                        
average loss         ...............................1.091e+00
average accuracy     ...............................8.219e-01
max accuracy         ...............................8.270e-01


## Ridge regression

In [38]:
# Ridge regression
lambda_ = 1e-7
scores, weights, loss_avg, score_avg = cross_validation(y, X_sigmoid, 'RR', {"lambda_": lambda_}, 
                                                        threshold = 0, k=k_fold, seed=0)
print_score(loss_avg, score_avg, scores, method='Ridge regression')

                      Ridge regression                      
average loss         ...............................1.092e+00
average accuracy     ...............................8.222e-01
max accuracy         ...............................8.280e-01


## Gradient decend

In [18]:
from costs import sigmoid
X_sigmoid = np.copy(X)
X_sigmoid = sigmoid(X_sigmoid)

In [39]:
# Gradient decend
max_iters = 100
gamma = 1e-2
scores, weights,loss_avg , score_avg = cross_validation(y,X_sigmoid, 'GD', 
                                                        {
                                                            "initial_w": initial_w,
                                                            "gamma": gamma,
                                                            "max_iters": max_iters,
                                                        },
                                                        threshold = 0,
                                                         k=3, seed=0)

print_score(loss_avg, score_avg, scores, method='Gradient decend')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
method                            step         loss        w0         w1    
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gradient descent                  0/99     4.584e-01 
gradient descent                 10/99     4.351e-01 
gradient descent                 20/99     4.266e-01 
gradient descent                 30/99     4.197e-01 
gradient descent                 40/99     4.139e-01 
gradient descent                 50/99     4.089e-01 
gradient descent                 60/99     4.045e-01 
gradient descent                 70/99     4.006e-01 
gradient descent                 80/99     3.971e-01 
gradient descent                 90/99     3.939e-01 
gradient descent                 99/99     3.913e-01 
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
method                            step         loss        w0         w1    
+++++++++++++++++++++

## Stochastic gradient decend

In [41]:
# Stochastic gradient decend
max_iters = 100
gamma = 1e-4
scores, weights,loss_avg , score_avg = cross_validation(y, X_sigmoid, 'SGD', 
                                                        {
                                                            "initial_w": initial_w,
                                                            "gamma": gamma,
                                                            "max_iters": max_iters,
                                                        },
                                                        threshold = 0,
                                                         k=5, seed=0)
print_score(loss_avg, score_avg, scores, method='Gradient decend')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
method                            step         loss        w0         w1    
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
stochastic gradient descent       0/99     4.985e-01 
stochastic gradient descent      10/99     5.051e-01 
stochastic gradient descent      20/99     4.949e-01 
stochastic gradient descent      30/99     4.945e-01 
stochastic gradient descent      40/99     4.919e-01 
stochastic gradient descent      50/99     4.893e-01 
stochastic gradient descent      60/99     4.866e-01 
stochastic gradient descent      70/99     4.772e-01 
stochastic gradient descent      80/99     4.734e-01 
stochastic gradient descent      90/99     4.701e-01 
stochastic gradient descent      99/99     4.695e-01 
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
method                            step         loss        w0         w1    
+++++++++++++++++++++

## Logistic regression

In [77]:
y_log = np.copy(y)
y_log[np.where(y_log[:]== -1)] = 0

max_iters = 800
gamma = 1e-6
# Logistic regression
scores, weights,loss_avg , score_avg = cross_validation(y_log, X_sigmoid, 'LR', 
                                                        {
                                                            "initial_w": initial_w,
                                                            "gamma": gamma,
                                                            "max_iters": max_iters,
                                                        },
                                                        threshold = 0.5,
                                                         k=2, seed=0)
print_score(loss_avg, score_avg, scores, method='Logistic regression')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
method                            step         loss        w0         w1    
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
logistic regression               0/799    6.491e-01  -1.449e-02 -1.144e-02
logistic regression              10/799    6.077e-01  -5.405e-03 -1.973e-02
logistic regression              20/799    5.909e-01  -2.400e-03 -2.751e-02
logistic regression              30/799    5.787e-01  1.176e-04  -3.262e-02
logistic regression              40/799    5.690e-01  2.314e-03  -3.583e-02
logistic regression              50/799    5.611e-01  4.280e-03  -3.764e-02
logistic regression              60/799    5.543e-01  6.072e-03  -3.837e-02
logistic regression              70/799    5.486e-01  7.727e-03  -3.825e-02
logistic regression              80/799    5.436e-01  9.271e-03  -3.745e-02
logistic regression              90/799    5.393e-01  1.072e-02  -3.610e-02
logistic 

logistic regression             210/799    5.131e-01  2.348e-02  3.693e-03 
logistic regression             220/799    5.117e-01  2.442e-02  7.612e-03 
logistic regression             230/799    5.103e-01  2.536e-02  1.157e-02 
logistic regression             240/799    5.090e-01  2.628e-02  1.556e-02 
logistic regression             250/799    5.078e-01  2.719e-02  1.957e-02 
logistic regression             260/799    5.067e-01  2.809e-02  2.361e-02 
logistic regression             270/799    5.056e-01  2.898e-02  2.765e-02 
logistic regression             280/799    5.045e-01  2.987e-02  3.170e-02 
logistic regression             290/799    5.035e-01  3.074e-02  3.576e-02 
logistic regression             300/799    5.026e-01  3.161e-02  3.982e-02 
logistic regression             310/799    5.016e-01  3.248e-02  4.388e-02 
logistic regression             320/799    5.007e-01  3.334e-02  4.793e-02 
logistic regression             330/799    4.999e-01  3.419e-02  5.198e-02 
logistic reg

## Regularized logistic regression

In [79]:
y_log = np.copy(y)
y_log[np.where(y_log[:]== -1)] = 0

max_iters = 500
gamma = 1e-6
lambda_ = 1e-5

# Regularized logistic regression
scores, weights,loss_avg , score_avg = cross_validation(y_log, X_sigmoid, 'RLR', 
                                                        {
                                                            "initial_w": initial_w,
                                                            "gamma": gamma,
                                                            "max_iters": max_iters,
                                                            "lambda_": lambda_
                                                        },
                                                        threshold = 0.6,
                                                         k=2, seed=0)
print_score(loss_avg, score_avg, scores, method='Regularized logistic regression')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
method                            step         loss        w0         w1    
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
regularized logistic regression    0/499    6.491e-01  -1.449e-02 -1.144e-02
regularized logistic regression   10/499    6.077e-01  -5.405e-03 -1.973e-02
regularized logistic regression   20/499    5.909e-01  -2.400e-03 -2.751e-02
regularized logistic regression   30/499    5.787e-01  1.176e-04  -3.262e-02
regularized logistic regression   40/499    5.690e-01  2.314e-03  -3.583e-02
regularized logistic regression   50/499    5.611e-01  4.280e-03  -3.764e-02
regularized logistic regression   60/499    5.544e-01  6.072e-03  -3.837e-02
regularized logistic regression   70/499    5.486e-01  7.727e-03  -3.825e-02
regularized logistic regression   80/499    5.436e-01  9.271e-03  -3.745e-02
regularized logistic regression   90/499    5.393e-01  1.072e-02  -3.610e-02

regularized logistic regression  499/499    4.888e-01  4.797e-02  1.182e-01 
              Regularized logistic regression               
average loss         ...............................4.879e-01
average accuracy     ...............................7.372e-01
max accuracy         ...............................7.392e-01
